In [35]:
import pandas as pd
import csv
import geopandas as gpd

In [47]:
population = pd.read_csv('../data/raw/population_for_suburb.csv',index_col=0)
population_pred_greater_melbourne= pd.read_csv('../data/raw/external_data/population_pred_greater_melbourne.csv',index_col=0)
population_pred_rest_vic= pd.read_csv('../data/raw/external_data/population_pred_rest_vic.csv',index_col=0)
properties_preporcessed_1=pd.read_csv('../data/raw/properties_preporcessed_1.csv',index_col=0)
SA2_2016=gpd.read_file('../data/raw/external_data/SA2_shape_2016/SA2_2016_AUST.shp')
SA2_2016=SA2_2016[SA2_2016['STE_NAME16'] == 'Victoria']


In [37]:
population_pred_greater_melbourne=population_pred_greater_melbourne.loc[2022:2025,['Annual population growth(e)']]/100 + 1
population_pred_rest_vic=population_pred_rest_vic.loc[2022:2025,['Annual population growth(e)']]/100 + 1

In [38]:
properties_preporcessed_1=properties_preporcessed_1[['SA2_MAIN16','LOC_PID']].copy()
properties_preporcessed_1=properties_preporcessed_1.groupby(['SA2_MAIN16','LOC_PID']).count()


properties_preporcessed_1=properties_preporcessed_1.reset_index()
properties_preporcessed_1.SA2_MAIN16=properties_preporcessed_1.SA2_MAIN16.astype('int')
properties_preporcessed_1

,SA2_MAIN16,LOC_PID
0,201011001,loc82f5b45a790d
1,201011001,loc8c852c59ffe5
2,201011001,loc98325a7e67bf
3,201011002,loc2c1864e3cbb6
4,201011002,loc76166a14c514
...,...,...
840,217041478,loce829f646f7ca
841,217041478,locf00a01fc543b
842,217041479,loc1f73672977ce
843,217041480,loc1f73672977ce


In [39]:
SA2_2016 = SA2_2016[['SA2_MAIN16','GCC_NAME16']]
SA2_2016 = SA2_2016[SA2_2016['GCC_NAME16'].isin(['Greater Melbourne','Rest of Vic.'])].copy()
SA2_2016 = SA2_2016.set_index('SA2_MAIN16').reset_index()
SA2_2016.SA2_MAIN16 = SA2_2016.SA2_MAIN16.astype('int')
SA2_2016

,SA2_MAIN16,GCC_NAME16
0,201011001,Rest of Vic.
1,201011002,Rest of Vic.
2,201011003,Rest of Vic.
3,201011004,Rest of Vic.
4,201011005,Rest of Vic.
...,...,...
457,217031476,Rest of Vic.
458,217041477,Rest of Vic.
459,217041478,Rest of Vic.
460,217041479,Rest of Vic.


In [40]:
df2=pd.merge(properties_preporcessed_1,SA2_2016)
df2=df2[['LOC_PID','GCC_NAME16']].copy()
df2

,LOC_PID,GCC_NAME16
0,loc82f5b45a790d,Rest of Vic.
1,loc8c852c59ffe5,Rest of Vic.
2,loc98325a7e67bf,Rest of Vic.
3,loc2c1864e3cbb6,Rest of Vic.
4,loc76166a14c514,Rest of Vic.
...,...,...
840,loce829f646f7ca,Rest of Vic.
841,locf00a01fc543b,Rest of Vic.
842,loc1f73672977ce,Rest of Vic.
843,loc1f73672977ce,Rest of Vic.


In [41]:
population=pd.merge(population,df2,)[['LOC_PID','suburb_population','GCC_NAME16']]

Greater_Melbourne=population[population['GCC_NAME16'].isin(['Greater Melbourne'])].copy()
Rest_of_Vic=population[population['GCC_NAME16'].isin(['Rest of Vic.'])].copy()

population

,LOC_PID,suburb_population,GCC_NAME16
0,loc0067a4549ed1,4897.000000,Rest of Vic.
1,loc00a9769647d7,26158.000000,Greater Melbourne
2,loc00d1503504f1,41928.000000,Greater Melbourne
3,loc00d1503504f1,41928.000000,Greater Melbourne
4,loc00d1503504f1,41928.000000,Greater Melbourne
...,...,...,...
2622,locff6258c8ea42,9003.705882,Greater Melbourne
2623,locff62fb6a898a,22420.000000,Greater Melbourne
2624,locffa1c8993b70,5488.609375,Greater Melbourne
2625,locffb43e78ab10,2133.333333,Rest of Vic.


In [42]:
population_pred_greater_melbourne

,Annual population growth(e)
2022,1.010101
2023,1.010101
2024,1.010101
2025,1.010101


In [43]:
Greater_Melbourne['2022']=Greater_Melbourne['suburb_population']*population_pred_greater_melbourne.loc[2022]['Annual population growth(e)']
Greater_Melbourne['2023']=Greater_Melbourne['2022']*population_pred_greater_melbourne.loc[2023]['Annual population growth(e)']
Greater_Melbourne['2024']=Greater_Melbourne['2023']*population_pred_greater_melbourne.loc[2024]['Annual population growth(e)']
Greater_Melbourne['2025']=Greater_Melbourne['2024']*population_pred_greater_melbourne.loc[2025]['Annual population growth(e)']
Greater_Melbourne

,LOC_PID,suburb_population,GCC_NAME16,2022,2023,2024,2025
1,loc00a9769647d7,26158.000000,Greater Melbourne,26422.222560,26689.114011,26958.701312,27231.011693
2,loc00d1503504f1,41928.000000,Greater Melbourne,42351.515692,42779.309284,43211.423986,43647.903443
3,loc00d1503504f1,41928.000000,Greater Melbourne,42351.515692,42779.309284,43211.423986,43647.903443
4,loc00d1503504f1,41928.000000,Greater Melbourne,42351.515692,42779.309284,43211.423986,43647.903443
5,loc00d1503504f1,41928.000000,Greater Melbourne,42351.515692,42779.309284,43211.423986,43647.903443
...,...,...,...,...,...,...,...
2620,locff0c4d75067f,8020.000000,Greater Melbourne,8101.010204,8182.838687,8265.493712,8348.983629
2622,locff6258c8ea42,9003.705882,Greater Melbourne,9094.652523,9186.517808,9279.311017,9373.041523
2623,locff62fb6a898a,22420.000000,Greater Melbourne,22646.464936,22875.217376,23106.280427,23339.677428
2624,locffa1c8993b70,5488.609375,Greater Melbourne,5544.049945,5600.050515,5656.616743,5713.754342


In [44]:
Rest_of_Vic['2022']=Rest_of_Vic['suburb_population']*population_pred_rest_vic.loc[2022]['Annual population growth(e)']
Rest_of_Vic['2023']=Rest_of_Vic['2022']*population_pred_rest_vic.loc[2023]['Annual population growth(e)']
Rest_of_Vic['2024']=Rest_of_Vic['2023']*population_pred_rest_vic.loc[2024]['Annual population growth(e)']
Rest_of_Vic['2025']=Rest_of_Vic['2024']*population_pred_rest_vic.loc[2025]['Annual population growth(e)']
Rest_of_Vic

,LOC_PID,suburb_population,GCC_NAME16,2022,2023,2024,2025
0,loc0067a4549ed1,4897.000000,Rest of Vic.,4946.464651,4996.428940,5046.897918,5097.876680
10,loc00e6e39d335b,1057.555556,Rest of Vic.,1068.237936,1079.028218,1089.927493,1100.936860
19,loc0115c25c528c,1106.000000,Rest of Vic.,1117.171718,1128.456281,1139.854829,1151.368513
21,loc01636b23f63e,1046.083333,Rest of Vic.,1056.649833,1067.323063,1078.104104,1088.994043
22,loc01699d7adbaa,1793.333333,Rest of Vic.,1811.447813,1829.745266,1848.227541,1866.896504
...,...,...,...,...,...,...,...
2567,locfa471b99ea58,1680.000000,Rest of Vic.,1696.969698,1714.110806,1731.425057,1748.914197
2607,locfdb55e6f8522,2227.111111,Rest of Vic.,2249.607185,2272.330490,2295.283323,2318.468000
2609,locfe8a70072c86,3830.000000,Rest of Vic.,3868.686872,3907.764517,3947.236885,3987.107961
2621,locff58d0167065,10487.000000,Rest of Vic.,10592.929302,10699.928587,10808.008673,10917.180466


In [45]:
pred = pd.concat([Greater_Melbourne,Rest_of_Vic]).drop_duplicates().drop(columns='GCC_NAME16', axis=1)
for year in ['2022', '2023', '2024', '2025']:
  pred[year] = pred[year].map(lambda x : round(x))
pred = pred.rename(columns={'2022' : 'population_proj_2022',
                            '2023' : 'population_proj_2023',
                            '2024' : 'population_proj_2024', 
                            '2025' : 'population_proj_2025'})
pred

,LOC_PID,suburb_population,population_proj_2022,population_proj_2023,population_proj_2024,population_proj_2025
1,loc00a9769647d7,26158.000000,26422,26689,26959,27231
2,loc00d1503504f1,41928.000000,42352,42779,43211,43648
11,loc00f0949ea0ad,39266.294118,39663,40064,40468,40877
20,loc01478eda4021,9284.850000,9379,9473,9569,9666
23,loc01c2784fcea2,4395.530303,4440,4485,4530,4576
...,...,...,...,...,...,...
2567,locfa471b99ea58,1680.000000,1697,1714,1731,1749
2607,locfdb55e6f8522,2227.111111,2250,2272,2295,2318
2609,locfe8a70072c86,3830.000000,3869,3908,3947,3987
2621,locff58d0167065,10487.000000,10593,10700,10808,10917


In [46]:
pred.to_csv("../data/raw/population_projection.csv")